In [1]:
import geopandas as gpd

from GDRT.raster.register_images import align_two_rasters
from GDRT.raster.registration_algorithms import cv2_feature_matcher


FIXED_FILENAME = "C:\\Users\\david\\Downloads\\orthomosaic_580.tif"
MOVING_FILENAME = "C:\\Users\\david\\Downloads\\orthomosaic_921.tif"
OUTFILE = "C:\\Users\\david\\scratch\\registration\orthomosaic_921_shifted.tif"
VECTOR_FILE = "C:\\Users\\david\\scratch\\registration\ROI.gpkg"

BUFFER_METERS = 50
TARGET_GSD = 0.25

# TODO Convert to the appropriate CRS
roi = gpd.read_file(VECTOR_FILE).to_crs(3310)
roi = roi.dissolve()

align_two_rasters(
    FIXED_FILENAME,
    MOVING_FILENAME,
    aligner_alg=cv2_feature_matcher,
    region_of_interest=roi,
    target_GSD=TARGET_GSD,
    vis_chips=True,
)

WindowError: width is required when providing negative indexes